In [8]:
import tensorflow as tf  
import sys  
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
#定义存储地址与名称
CNN = '/home/aa/17b-yangxueshu-2015/FashionMNIST_Challenge/code/cnn'
cnn = 'mnist'

In [7]:
#数据处理
def dataset(data_type='train', batch_size=10):
    # 创建队列保护输入文件列表
    filename_queue = tf.train.string_input_producer(['data/data_tfrecord/' + data_type + '.tfrecords'])  
    # 读取并解析一个tfrecord
    reader = tf.TFRecordReader()  
    _, serialized_example = reader.read(filename_queue)
    example = tf.parse_single_example(serialized_example, features={
        'f1': tf.FixedLenFeature([], tf.string),
        'f2': tf.FixedLenFeature([], tf.string)})
    # 将字符串分别解析成标签和图像对应的像素组（并重组）
    image = tf.decode_raw(example['f1'], tf.float32)
    label = tf.decode_raw(example['f2'], tf.float64)
    image = tf.reshape(image, [28, 28])
    label= tf.reshape(label,[1,10])
    
    img_batch, l_batch = tf.train.shuffle_batch(
        [image, label],
        batch_size=batch_size,
        capacity= (3 * batch_size),
        min_after_dequeue=batch_size)
    return img_batch, l_batch

In [5]:
def weight_variable(shape):  
    # 使用截断正态分布生成卷积核
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  
def bias_variable(shape):  
    # 使用relu激活函数，用一个正偏置值较准
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
    #定义卷积层
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    # 定义池化层
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  

In [9]:
#定义模型文件所在的文件夹，若不存在则自动创建
ckpt = tf.train.latest_checkpoint(CNN)
if not ckpt:
    if not os.path.exists(CNN):
        os.mkdir(CNN)

In [10]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# 生成
W = tf.Variable(tf.zeros([784,10]))  
b = tf.Variable(tf.zeros([10]))  

# 第一层 先卷积再池化，卷积核尺寸为5*5, 通道数为1，输出通道为32，池化输出后应为14*14*32
W_conv1 = weight_variable([5, 5, 1, 32])  
b_conv1 = bias_variable([32])  
# 格式转换
x_image = tf.reshape(x, [-1, 28, 28, 1])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  

# 第二层 先卷积再池化，卷积核尺寸为5*5, 输入通道为32，输出通道为64，池化输出后应为7*7*64
W_conv2 = weight_variable([5, 5, 32, 64])  
b_conv2 = bias_variable([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
# 此时图片格式简化为7*7 进入第三层全连接层,输入维数7*7*64, 输出维数为1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])  
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
keep_prob = tf.placeholder("float")  
# 使用drop out防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

# 第四层，输入1024维，输出10维，即0~9分类
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])   
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
  
# 损失函数，交叉熵 
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
# 使用adam优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 
# 准确度计算
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  

# 数据更新
test_acc_sum = tf.Variable(0.0)
batch_acc = tf.placeholder(tf.float32)
new_test_acc_sum = tf.add(test_acc_sum, batch_acc)
update = tf.assign(test_acc_sum, new_test_acc_sum)

#存储
saver=tf.train.Saver(max_to_keep=2)

# 变量初始化，此为训练用代码
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(CNN,sess.graph)
    ckpt = tf.train.latest_checkpoint(CNN)
    step=0
    img, lable = dataset(batch_size=500)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
    for i in range(500):  
        X, Y = sess.run([img[i], lable[i]])
        if i%100 == 0:  
            train_accuracy = accuracy.eval(feed_dict={  
                x:X, y_: Y, keep_prob: 1.0})  
            print("step %d, training accuracy %.3f"%(i, train_accuracy))
            ckptname=os.path.join(CNN, cnn)
            saver.save(sess,ckptname,global_step=i)
        train_step.run(feed_dict={x: X, y_: Y, keep_prob: 0.5}) 

INFO:tensorflow:Restoring parameters from /home/aa/17b-yangxueshu-2015/FashionMNIST_Challenge/code/cnn/mnist-300
step 0, training accuracy 0.800
step 100, training accuracy 0.860
step 200, training accuracy 0.860
step 300, training accuracy 0.840
step 400, training accuracy 0.840
